# **Data Aquistion**


This notebook downloads the required data for state-of-the-art LCZ Classification:

1. Setup the project by defining the city of interest
2. City boundary polygon download from OpenStreetMap 
3. Sentinel-2 imagery download using AWS S3
4. ALOS DSM 30m download from Google Earth Engine
5. Impervious Surface Area download from Google Earth Engine 
6. Tree Canopy Height download from Google Earth Engine
7. Building Height download from NRCan

## **1. Project Setup**


This code block imports the necessery Python libraries

### 1.1. Import Libraries

In [2]:
%load_ext autoreload
%autoreload 

import sys
import os

# Add the module's parent directory to sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from lcz_classification.config import  *
from lcz_classification.dataset import get_matching_scenes, download_tiles, ee_download, fetch_metadata
from lcz_classification.util import tiles_from_bbox
import geopandas as gpd
import matplotlib.pyplot as plt
import ee
import numpy as np
import pandas as pd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### 1.2. Setup Directories

In [141]:

study_area=fetch_metadata('STUDY_AREA')
bounds=list(study_area.total_bounds)

s2_tiles=fetch_metadata('S2_TILES')

tiles_gdf=tiles_from_bbox(bounds, tile_dims=(2,2)).sort_values('tile_id')

# 2.1. Get Study Area

In [ ]:
study_area.explore(
                style=dict(
                    fill = False,
                    color='red'
                ),
)

### 1.3 Authenticate Google Earth Engine

In [8]:
if ee.Authenticate() == True:
    ee.Initialize(project=EE_PROJECT_NAME)    
else:
    ee.Authenticate(force = True)
    ee.Initialize(project=EE_PROJECT_NAME)    

# **3. Sentinel-2 Download**

Using download_tiles function, Sentinel-2 data is downloaded from the AWS S3 Bucket using boto3. The city boundary polygon and data range are used to derive matching scenes withiin the sentinel-2 database.

https://registry.opendata.aws/sentinel-2/


In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


# Function to get least-cloudy image for one tile
# Function to get least-cloudy image for one tile
def get_least_cloudy(col,tile_id):
    filtered = (
        col
        .filter(ee.Filter.eq('MGRS_TILE', tile_id))
        .sort('CLOUDY_PIXEL_PERCENTAGE')  # least cloudy first
    )
    return ee.Image(filtered.first())
    


In [ ]:
import geemap

def ee_least_cloudy(col, bbox_geom):
        least_cloudy_images=list()
        ee_tile_ids=sorted(col.aggregate_array('MGRS_TILE').distinct().getInfo())
        least_cloudy_images=list()

        for ee_tile in ee_tile_ids:
            
            tile_filter=ee.Filter.eq('MGRS_TILE',ee_tile)
            tile_cloud_percentages=col.filter(tile_filter).aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo()
            least_cloudy_val = min(tile_cloud_percentages)
            least_cloudy_image=col.filter(tile_filter).filter(ee.Filter.eq('CLOUDY_PIXEL_PERCENTAGE', least_cloudy_val)).first().clip(bbox_geom)
            image_id = least_cloudy_image.getInfo()['id']
            print(f"- Lowest Cloud Percentage in Image {image_id.split('/')[-1]} : {round(least_cloudy_val * 100, 1)}%")

            least_cloudy_images.append(least_cloudy_image)


        return ee.ImageCollection(ee.List(least_cloudy_images))


# iterate throught tile geometries generated with tiles_from_bbox()
for idx, tile in tiles_gdf.iloc.iterrows():

     tile_id = tile.tile_id
     print(f"Retrieving ImageCollection for Tile {tile_id}")
     print(f"===================================================")
     ee_geom= ee.Geometry.Rectangle(tile.geometry.bounds)
     START_DATE , END_DATE= ('2021-08-01' , '2021-08-30')

     # Get ImageCollection filtered by date range, cloud percentage and first tile
     col=(ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
          .filterBounds(ee_geom)
          .filterDate(START_DATE, END_DATE)
          )
     
     features = col.getInfo()['features']
     print(f"- Number of Features: {len(features)}")
     # Retrieve Least Cloudy Images from the Collection
     least_cloudy_col = ee_least_cloudy(col, ee_geom)

     # Mosaic Images of Least Cloudy Collection and Clip to Bounds     
     image=ee.Image(least_cloudy_col.mosaic())
     # image = mask_s2_clouds(image)
     
     # Get Matching Tile IDs
     # break
     print(f'Downloading Images for Tile: {tile_id}')
     # Iterate through target S2 bands and export
     for band in S2_TARGET_BANDS:
          out_dir = f"{S2_RAW}/T{tile_id}"
          filename=f"{out_dir}/{band}.tif"
          if os.path.exists(out_dir) == False:
               os.mkdir(out_dir)
               
          if os.path.exists(filename) == False:
               print(f"- Downloading ee.Image for tile {tile_id} and Band {band}")
               ee_band=band.replace('0','')
               # Export Image using geemap
               geemap.ee_export_image(
                         image.select(ee_band),
                         filename=filename,
                         scale=CELL_RESOLUTION,
                         file_per_band=False,
                         crs='EPSG:4326', 
                         region = ee_geom
                    )
          else:
               print(f"- GeoTIFF for Tile {tile_id} Band {band} already downloaded")

     print(f"===================================================")
     

Retrieving ImageCollection for Tile A2
- Number of Features: 30
- Lowest Cloud Percentage in Image 20210824T160829_20210824T161729_T17TNH : 16.3%
- Lowest Cloud Percentage in Image 20210824T160829_20210824T161729_T17TNJ : 28.9%
- Lowest Cloud Percentage in Image 20210824T160829_20210824T161729_T17TPH : 162.2%
- Lowest Cloud Percentage in Image 20210824T160829_20210824T161729_T17TPJ : 29.5%
- GeoTIFF for Tile A2 Band B02 already downloaded
- GeoTIFF for Tile A2 Band B03 already downloaded
- GeoTIFF for Tile A2 Band B04 already downloaded
- GeoTIFF for Tile A2 Band B05 already downloaded
- GeoTIFF for Tile A2 Band B06 already downloaded
- GeoTIFF for Tile A2 Band B07 already downloaded
- Downloading ee.Image for tile A2 and Band B8A
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\sentinel2\TA2\B8A.tif
- Downloading ee.Image for tile A2 and Band B11
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Class

In [273]:

# Select bands for visualization (e.g., RGB)
vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],  # RGB
}

# Create an interactive map
x,y = ee_geom.centroid().getInfo()['coordinates']
Map = geemap.Map(center=(y,x), zoom=10)
Map.addLayer(image, vis_params, 'Sentinel-2 RGB')
Map.addLayer(ee_geom, {}, 'AOI')
Map

Map(center=[43.54101885777565, -79.79510090012732], controls=(WidgetControl(options=['position', 'transparent_…

## **4. DSM from ALOS 30m**

In [ ]:
# Download ALOS DSM Data from Google Earth Engine

ee_download(
            asset_id="JAXA/ALOS/AW3D30/V3_2",
            bands=["DSM"],
            bbox = bounds, 
            output_dir=DSM_RAW,
            scale=CELL_RESOLUTION,
            tile_dims=(2,1)
            )



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
IMAGE_COLLECTION
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\alos_dsm_30m\N043W079_DSM_A1.tif
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\alos_dsm_30m\N043W079_DSM_B1.tif
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\alos_dsm_30m\N043W080_DSM_A1.tif
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\alos_dsm_30m\N043W080_DSM_B1.tif
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\alos_dsm_30m\N043W081_DSM_A1.tif
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\alos_dsm_30m\N043W081_DSM_B1.tif
Generating URL ...
Please wait ...
Data downloade

## **5. Impervious Surface Area**

In [21]:
# # Download Impervious Surface Area from Google Earth Engine

# ee_download(
#             asset_id="projects/sat-io/open-datasets/GISD30_1985_2020",
#             bands=["b1"],
#             bbox =bounds, 
#             output_dir=IS_RAW,
#             scale=CELL_RESOLUTION,
#             tile_dims=tile_dims
#             )

IMAGE
Generating URL ...
An error occurred while downloading.
Total request size (74553936 bytes) must be less than or equal to 50331648 bytes.


## **6. Tree Canopy Height**

In [26]:
# Download Canopy Height from Google Earth Engine


ee_download(
            asset_id="users/nlang/ETH_GlobalCanopyHeight_2020_10m_v1",
            bands=["b1"],
            bbox = bounds, 
            output_dir=CH_RAW,
            scale=10,
            tile_dims=(4,4)
            )

IMAGE
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\canopy_height\ETH_GlobalCanopyHeight_2020_10m_v1_b1_A4.tif
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\canopy_height\ETH_GlobalCanopyHeight_2020_10m_v1_b1_A3.tif
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\canopy_height\ETH_GlobalCanopyHeight_2020_10m_v1_b1_A2.tif
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\canopy_height\ETH_GlobalCanopyHeight_2020_10m_v1_b1_A1.tif
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\canopy_height\ETH_GlobalCanopyHeight_2020_10m_v1_b1_B4.tif
Generating URL ...
Please wait ...
Data downloaded to d:\GeoAI\projects\LCZ_Classification\data\toronto\raw\canopy_height\ETH_GlobalCanopyHeight_2020_10m_v1_b1

## **7. Building Height Data**

Download building height data for British Columbia from the NRCan Website- https://ftp.maps.canada.ca/pub/nrcan_rncan/extraction/auto_building/fgdb/

In [28]:
import requests
from bs4 import BeautifulSoup
import os
from tqdm import tqdm

# Base URL of the directory
BASE_URL = "https://ftp.maps.canada.ca/pub/nrcan_rncan/extraction/auto_building/fgdb/"
PROVINCE = "ON"
# Create a session
session = requests.Session()

def get_links(url, province):
    print(f"Fetching links from {url}")
    response = session.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')
    links = []
    for link in soup.find_all('a'):
        href = link.get('href')
        if href and province in href and href.endswith('.zip'):
            full_url = url + href
            links.append(full_url)
    return links

def download_file(url, dest_folder="."):
    local_filename = os.path.join(dest_folder, url.split("/")[-1])
    print(f"Downloading: {url}")
    with session.get(url, stream=True) as r:
        r.raise_for_status()
        total = int(r.headers.get('content-length', 0))
        with open(local_filename, 'wb') as f, tqdm(
            desc=local_filename,
            total=total,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    bar.update(len(chunk))
    print(f"Saved to {local_filename}")

if __name__ == "__main__":
    links = get_links(BASE_URL, PROVINCE)
    print(f"Found {len(links)} {PROVINCE}-related files.")
    for link in links:
        download_file(link, dest_folder=BH_RAW)


Fetching links from https://ftp.maps.canada.ca/pub/nrcan_rncan/extraction/auto_building/fgdb/
Found 29 ON-related files.
Downloading: https://ftp.maps.canada.ca/pub/nrcan_rncan/extraction/auto_building/fgdb/Autobuilding_ON_2016-17-Peterborough_fgdb.zip


../data/toronto/raw/building_height\Autobuilding_ON_2016-17-Peterborough_fgdb.zip: 100%|██████████| 33.9M/33.9M [00:17<00:00, 2.01MB/s]


Saved to ../data/toronto/raw/building_height\Autobuilding_ON_2016-17-Peterborough_fgdb.zip
Downloading: https://ftp.maps.canada.ca/pub/nrcan_rncan/extraction/auto_building/fgdb/Autobuilding_ON_2017-LakeErie_fgdb.zip


../data/toronto/raw/building_height\Autobuilding_ON_2017-LakeErie_fgdb.zip: 100%|██████████| 94.5M/94.5M [00:21<00:00, 4.70MB/s]


Saved to ../data/toronto/raw/building_height\Autobuilding_ON_2017-LakeErie_fgdb.zip
Downloading: https://ftp.maps.canada.ca/pub/nrcan_rncan/extraction/auto_building/fgdb/Autobuilding_ON_BRAMPTON_fgdb.zip


...

Saved to ../data/toronto/raw/building_height\Autobuilding_QC_2009_701-09-PONTIAC_MTM09_fgdb.zip
Downloading: https://ftp.maps.canada.ca/pub/nrcan_rncan/extraction/auto_building/fgdb/Autobuilding_QC_2014_600014-35-PONTIAC_MTM09_fgdb.zip


../data/toronto/raw/building_height\Autobuilding_QC_2014_600014-35-PONTIAC_MTM09_fgdb.zip: 100%|██████████| 1.86M/1.86M [00:00<00:00, 3.28MB/s]


Saved to ../data/toronto/raw/building_height\Autobuilding_QC_2014_600014-35-PONTIAC_MTM09_fgdb.zip
Downloading: https://ftp.maps.canada.ca/pub/nrcan_rncan/extraction/auto_building/fgdb/Autobuilding_QC_VILLE_MONTREAL_fgdb.zip


../data/toronto/raw/building_height\Autobuilding_QC_VILLE_MONTREAL_fgdb.zip: 100%|██████████| 29.7M/29.7M [00:05<00:00, 6.10MB/s]

Saved to ../data/toronto/raw/building_height\Autobuilding_QC_VILLE_MONTREAL_fgdb.zip
